# Tanpopo 表面付着物 ViT CrossValidation

In [ ]:
!pip install transformers
!pip list | grep transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 6.6 MB/s 
     |████████████████████████████████| 182 kB 65.8 MB/s 
     |████████████████████████████████| 7.6 MB 68.3 MB/s 
transformers                  4.25.1


In [ ]:
# Google Colab マウント
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive
import os
os.chdir('/content/drive/MyDrive/Tanpopo')

Mounted at /content/drive
/content/drive/MyDrive


In [ ]:
from PIL import Image
import requests
import matplotlib.pyplot as plt

import torch
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets, models, transforms
import torch.utils.data as data

import glob
import time
import copy
from PIL import Image

In [ ]:
#画像サイズがが704x480 #88x60
img_size = 224

#class_names = ['1Sputter', '2Fiber', '3Block', '4Bar', '5AGFragment']
class_num = 5

# 標準化
mean = (0.5, 0.5, 0.5)
std = (0.5, 0.5, 0.5)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#torch.set_default_tensor_type('torch.cuda.FloatTensor')

In [ ]:
batch_size =  "16" #@param[8, 16, 32, 64, 128, 256]
batch_size = int(batch_size)

epochs = "25" #@param[5, 8, 10, 15, 20, 22, 25, 27, 29, 30, 31, 32, 33, 35, 45, 60, 120]
epochs = int(epochs)

### 関数、クラスの定義

In [ ]:
import random
from sklearn.model_selection import train_test_split
def make_filepath_list(folderpath, phase='train'):
    """
    ファイルのパスを格納したリストを返す
    """
    # .DS_Storeが最初に読み込まれる
    file_list = []
    files_list = []
    class_names = []

    for index, top_dir in enumerate(sorted(os.listdir(folderpath))):
        file_dir = os.path.join(folderpath, top_dir)
        file_list = glob.glob(file_dir + '/*bmp')

        if top_dir != '.DS_Store':
            class_names.append(top_dir)
            files_list += [os.path.join(folderpath, top_dir, file).replace('\\', '/') for file in file_list]
                                                            
    return files_list, class_names

In [ ]:
# train_list1 = os.listdir('/content/drive/MyDrive/Tanpopo/TrainingData11')
# test_list1 = os.listdir('/content/drive/MyDrive/Tanpopo/TestData11')
# print('trian : ', train_list1)
# print('trian : ', sorted(train_list1))
# print('test : ', test_list1)
# print('test : ', sorted(test_list1))

In [ ]:
class ImageTransform(object):
    """
    画像の前処理
    """
    def __init__(self, resize, mean, std):
        self.data_transform = {
            'train': transforms.Compose([
                # データオグメンテーション
                transforms.Resize(256), #リサイズ
                transforms.CenterCrop(resize), #切り取り
                transforms.RandomRotation(45), #ランタムに回転
                transforms.ColorJitter(), #ランダムに明るさ、コントラスト、彩度、色相を変化
                transforms.RandomHorizontalFlip(), #ランダムに左右(水平)反転
                transforms.RandomVerticalFlip(), #ランダムに上下(垂直)反転
                transforms.ToTensor(),
                transforms.Normalize(mean, std), #zcaと交換？
                #ZCA whitening追加する
            ]),
            'valid': transforms.Compose([
                transforms.Resize(256),
                transforms.CenterCrop(resize),
                transforms.ToTensor(),
                transforms.Normalize(mean, std),
            ]),
            'test': transforms.Compose([
                transforms.Resize(256),
                transforms.CenterCrop(resize),
                transforms.ToTensor(),
                transforms.Normalize(mean, std),
            ])
        }

    def __call__(self, img, phase='train'):
        return self.data_transform[phase](img)

In [ ]:
from torchvision.io import read_image

class SurfaceObjectDataset(data.Dataset):
    """
    表面付着物のDatasetクラス
    PyTorchのDatasetクラスを継承
    """
    def __init__(self, file_list, classes, transform=None, phase='train'):
        #super().__init__()
        self.file_list = file_list
        self.transform = transform
        self.classes = classes
        self.phase = phase

        #self.img = None
        #self.label = None

    def __len__(self):
        """
        画像の枚数を返す
        """
        return len(self.file_list)

    def __getitem__(self, index):
        """
        前処理した画像データのTensor形式のデータとラベルを取得
        """
        # 指定したindexの画像を読み込む
        img_path = self.file_list[index]
        img = Image.open(img_path)
        #img = read_image(img_path)

        # 画像ラベルをファイル名から抜き出す
        label = self.file_list[index].split('/')[6][:11]

        # ラベル名を数値に変換
        label = self.classes.index(label)
        #print(label, end=' ')

        # 画像の前処理を実施
        if self.transform is not None:
            img_transformed = self.transform(img, self.phase)
        
        #self.img = img_transformed
        #elf.label = label

        return img_transformed, label

In [ ]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))

        # 各エポックには訓練フェーズと検証フェーズがあります
        for phase in ['train', 'valid']:
            if phase == 'train':
                model.train()  # モデルを訓練モードに設定します
            else:
                model.eval()   # モードを評価するモデルを設定します

            running_loss = 0.0
            running_corrects = 0

            # データをイレテートします
            ##i = 0
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # パラメータの勾配をゼロにします
                optimizer.zero_grad()

                # 順伝播
                # 訓練の時だけ、履歴を保持します
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # 訓練の時だけ逆伝播＋オプティマイズを行います
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # 損失を計算します
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

                # # TensorBoardで可視化
                # if i % 8 == 7: # every 8 mini-batches
                #     if phase == 'train':
                #         writer.add_scalar('training loss', running_loss / 8,
                #                           epoch * len(dataloaders_dict[phase]) + i)
                #         writer.add_figure('Train predictions vs. actuals',
                #             plot_classes_preds(model, inputs, labels),
                #             global_step=epoch * len(dataloaders_dict[phase]) + i)
                #     elif phase == 'valid':
                #         writer.add_scalar('training loss', running_loss / 8,
                #                           epoch * len(dataloaders_dict[phase]) + i)
                #         writer.add_figure('Validation predictions vs. actuals',
                #             plot_classes_preds(model, inputs, labels),
                #             global_step=epoch * len(dataloaders_dict[phase]) + i)
                        
                # i += 1
                        

            # if phase == 'train':
            #     scheduler.step()

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print(' {} Loss: {:.4f} Acc: {:.4f} '.format(
                phase, epoch_loss, epoch_acc), end='\t')

            # モデルをディープ・コピーします
            if phase == 'valid' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()
        #print('-' * 10)

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # ベストモデルの重みをロードします
    model.load_state_dict(best_model_wts)
    return model, best_acc

In [ ]:
from sklearn.metrics import confusion_matrix
import numpy as np

#正解
def class_accuracy(label, conf_mat):
    return (conf_mat[label][label] + (np.sum(conf_mat) - (np.sum(conf_mat[:, label])+np.sum(conf_mat[label])-conf_mat[label][label]))) / np.sum(conf_mat)
    
#精度(適合率)
def class_precision(label, conf_mat):
    return conf_mat[label][label] / np.sum(conf_mat[label])

#再現率
def class_recall(label, conf_mat):
    return conf_mat[label][label] / np.sum(conf_mat[:, label])

# テスト結果を返す
def test_model(model, test_dataloaders):
    labels_sum = None
    predicted_sum = None

    with torch.no_grad():
        for data in test_dataloaders:
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)

            #outputs = nn.Softmax(dim=1)(outputs)

            _, predicted = torch.max(outputs, 1)

            if labels_sum is None:
                labels_sum = labels
                predicted_sum = predicted
            else:
                labels_sum = torch.cat([labels_sum, labels], dim=0)
                predicted_sum = torch.cat([predicted_sum, predicted], dim=0)

    #混同行列
    labels_sum = labels_sum.cpu()
    predicted_sum = predicted_sum.cpu()
    conf_mat = None
    Accuracy = []
    Precision = []
    Recall = []

    conf_mat = confusion_matrix(labels_sum, predicted_sum)

    for i in range(class_num):
        Accuracy = np.append(Accuracy, class_accuracy(i, conf_mat)*100)
        Precision = np.append(Precision, class_precision(i, conf_mat)*100)
        Recall = np.append(Recall, class_recall(i, conf_mat)*100)

    return conf_mat, Accuracy, Precision, Recall

### モデルの作成

In [ ]:
from transformers import ViTFeatureExtractor, ViTModel
# ファインチューニングされたモデルをロードして使う場合はViTForImageClassificationですぐに分類問題に適用できるようですが、
# 今回はファインチューニングの実装のところからも行いたいので、こちらは使いません。
# from transformers import ViTForImageClassification

# 前処理用クラス
feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224-in21k')

# モデル本体
# 順伝播時の出力にAttentionの結果もほしいときはoutput_attentions=Trueを指定する。
vit_model = ViTModel.from_pretrained('google/vit-base-patch16-224-in21k', output_attentions=True)

Downloading:   0%|          | 0.00/160 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/502 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/346M [00:00<?, ?B/s]

In [ ]:
print(vit_model)

ViTModel(
  (embeddings): ViTEmbeddings(
    (patch_embeddings): ViTPatchEmbeddings(
      (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    )
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (encoder): ViTEncoder(
    (layer): ModuleList(
      (0): ViTLayer(
        (attention): ViTAttention(
          (attention): ViTSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
          )
          (output): ViTSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
          )
        )
        (intermediate): ViTIntermediate(
          (dense): Linear(in_features=768, out_features=3072, bias=True)
          (intermediate_act_fn): GELUActivation()
      

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class ViTNet(nn.Module):
    def __init__(self, pretrained_vit_model, class_num):
        super(ViTNet, self).__init__()
        self.vit = pretrained_vit_model
        self.fc = nn.Linear(768, class_num)

    def _get_cls_vec(self, states): # fine tuningで
        return states['last_hidden_state'][:, 0, :]

    def forward(self, input_ids):
        states = self.vit(input_ids)
        states = self._get_cls_vec(states)
        states = self.fc(states)
        return states
    
model_ViT = ViTNet(vit_model, class_num)
#model_ViT.to(device)

In [ ]:
# まず全パラメータを勾配計算Falseにする
for param in model_ViT.parameters():
    param.requires_grad = False

# 追加したクラス分類用の全結合層を勾配計算ありに変更
for param in model_ViT.fc.parameters():
    param.requires_grad = True

optimizer = optim.AdamW([
    {'params': model_ViT.fc.parameters(), 'lr': 0.001, 'weight_decay': 0.5}
    #optim.AdamW(batchNorm_params_to_update, lr=0.001, weight_decay=0.001)
])
# optimizer = optim.RAdam([
#     {'params': model_ViT.fc.parameters(), 'lr': 0.001, 'weight_decay': 0.1}
# ])

criterion = nn.CrossEntropyLoss()

### 学習

In [ ]:
from torch.utils.data.dataset import Subset
from sklearn.model_selection import KFold
from  torch.utils.data import Dataset

# 訓練、検証データへのファイルパスを格納したリストを取得
train_file_list, class_names = make_filepath_list('/content/drive/MyDrive/Tanpopo/TrainingData11', 'train')
#print('train_file_list: ', train_file_list)
#print('class_names: ', class_names)
class_num = len(class_names) # 5

dataset = SurfaceObjectDataset(
        file_list = train_file_list, classes = class_names,
        transform = ImageTransform(img_size, mean, std),
        phase = 'train')

# テストデータ
test_file_list, class_names_test = make_filepath_list('/content/drive/MyDrive/Tanpopo/TestData11', 'test')
#print('test_file_list : ', test_file_list)
#print('class_names_test : ', class_names_test)

# Datasetの作成
test_dataset = SurfaceObjectDataset(
    file_list = test_file_list, classes = class_names_test,
    transform = ImageTransform(img_size, mean, std),
    phase = 'test')
# Dataloaderの作成
test_dataloader = data.DataLoader(
    test_dataset, batch_size = int(batch_size/2), shuffle=False)
dataloaders_dict = {'test': test_dataloader}


# K-Fold 交差検証 
kf = KFold(n_splits=5, shuffle=True, random_state=1) # 5回検証

scores = []
test_accuracy = []
test_precision = []
test_recall = []
conf_mat = []

PATH = '/content/drive/MyDrive/Tanpopo/model_ViT_weights.pth'
model_ViT = model_ViT.to(device)
model_ViT.load_state_dict(torch.load(PATH)) # 学習前: 学習済みの重みを使う

for _fold, (train_index, valid_index) in enumerate(kf.split(train_file_list)):
    # Datasetの作成
    train_dataset = Subset(dataset, train_index)
    valid_dataset = Subset(dataset, valid_index)
    # Dataloaderの作成
    train_dataloader = data.DataLoader(
        train_dataset, batch_size = batch_size, shuffle=True)
    valid_dataloader = data.DataLoader(
        valid_dataset, batch_size = int(batch_size/2), shuffle=False)
    
    dataloaders_dict['train'] = train_dataloader
    dataloaders_dict['valid'] = valid_dataloader

    print('-'*5, end='')
    print(_fold+1, end='')
    print('-'*5)
    model_ViT = model_ViT.to(device)
    model_ViT, score = train_model(model_ViT, dataloaders_dict, criterion, optimizer, num_epochs=epochs)
    scores.append(score.to('cpu'))

    conf, test_acc, test_prec, test_rec = test_model(model_ViT, dataloaders_dict['test'])

    test_accuracy.append(test_acc)
    test_precision.append(test_prec)
    test_recall.append(test_rec)
    conf_mat.append(conf)

-----1-----
Epoch 0/24
 train Loss: 0.1634 Acc: 0.9643 	 valid Loss: 0.1205 Acc: 0.9796 	
Epoch 1/24
 train Loss: 0.1608 Acc: 0.9490 	 valid Loss: 0.1411 Acc: 1.0000 	
Epoch 2/24
 train Loss: 0.1637 Acc: 0.9796 	 valid Loss: 0.1250 Acc: 0.9796 	
Epoch 3/24
 train Loss: 0.1659 Acc: 0.9643 	 valid Loss: 0.1427 Acc: 0.9796 	
Epoch 4/24
 train Loss: 0.1560 Acc: 0.9745 	 valid Loss: 0.1282 Acc: 0.9796 	
Epoch 5/24
 train Loss: 0.1737 Acc: 0.9643 	 valid Loss: 0.0973 Acc: 0.9796 	
Epoch 6/24
 train Loss: 0.1554 Acc: 0.9592 	 valid Loss: 0.1481 Acc: 0.9796 	
Epoch 7/24
 train Loss: 0.1427 Acc: 0.9745 	 valid Loss: 0.1250 Acc: 0.9796 	
Epoch 8/24
 train Loss: 0.1493 Acc: 0.9643 	 valid Loss: 0.1218 Acc: 1.0000 	
Epoch 9/24
 train Loss: 0.1553 Acc: 0.9796 	 valid Loss: 0.2036 Acc: 0.9388 	
Epoch 10/24
 train Loss: 0.1451 Acc: 0.9694 	 valid Loss: 0.1232 Acc: 0.9796 	
Epoch 11/24
 train Loss: 0.1640 Acc: 0.9592 	 valid Loss: 0.0984 Acc: 1.0000 	
Epoch 12/24
 train Loss: 0.1163 Acc: 0.9898 	 vali

In [ ]:
# Validation 
print('ViT Result')
print(f'Validation Accuracy: 平均 {np.mean(scores)*100:.1f} 標準偏差 {np.std(cores)*100:.4f}')

# Test
print('Test results:')
print(class_names)

tmp_list = [] # Accuracy
print('Accuracy:', end='\t')
for i in range(class_num):
    tmp_list = [r[i] for r in test_accuracy]
    means = np.mean(tmp_list)
    stds = np.std(tmp_list)
    print(f'{np.mean(tmp_list):.1f}±{np.std(tmp_list):.2f}', end='\t')

tmp_list = [] # Precision
print('\nPrecision:', end='\t')
for i in range(class_num):
    tmp_list = [r[i] for r in test_precision]
    means = np.mean(tmp_list)
    stds = np.std(tmp_list)
    print(f'{np.mean(tmp_list):.1f}±{np.std(tmp_list):.2f}', end='\t')

tmp_list = [] # Recall
print('\nRecall:', end='\t')
for i in range(class_num):
    tmp_list = [r[i] for r in test_recall]
    means = np.mean(tmp_list)
    stds = np.std(tmp_list)
    print(f'{np.mean(tmp_list):.1f}±{np.std(tmp_list):.2f}', end='\t')

print('\nConf Matrix:') # 混同行列
for i in range(class_num):
    print([r[i] for r in conf_mat])

ViT Result
Validation Accuracy: 平均 99.6 分散 0.0067
Test results:
['1Sputter', '2Fiber', '3Block', '4Bar', '5AGFragment']
Accuracy:	99.8±0.38	97.9±0.38	97.1±0.00	99.0±0.00	96.2±0.00	
Precision:	99.0±1.90	100.0±0.00	90.5±0.00	95.2±0.00	90.5±0.00	
Recall:	100.0±0.00	90.5±1.52	95.0±0.00	100.0±0.00	90.5±0.00	
Conf Matrix:
[array([21,  0,  0,  0,  0]), array([21,  0,  0,  0,  0]), array([20,  1,  0,  0,  0]), array([21,  0,  0,  0,  0]), array([21,  0,  0,  0,  0])]
[array([ 0, 21,  0,  0,  0]), array([ 0, 21,  0,  0,  0]), array([ 0, 21,  0,  0,  0]), array([ 0, 21,  0,  0,  0]), array([ 0, 21,  0,  0,  0])]
[array([ 0,  0, 19,  0,  2]), array([ 0,  0, 19,  0,  2]), array([ 0,  0, 19,  0,  2]), array([ 0,  0, 19,  0,  2]), array([ 0,  0, 19,  0,  2])]
[array([ 0,  1,  0, 20,  0]), array([ 0,  1,  0, 20,  0]), array([ 0,  1,  0, 20,  0]), array([ 0,  1,  0, 20,  0]), array([ 0,  1,  0, 20,  0])]
[array([ 0,  1,  1,  0, 19]), array([ 0,  1,  1,  0, 19]), array([ 0,  1,  1,  0, 19]), array([ 0,

In [ ]:
PATH = '/content/drive/MyDrive/Tanpopo/model_ViT_weights.pth'
#torch.save(model_ViT.state_dict(), PATH) # 重みを保存